In [1]:
!pip install datasets torch transformers evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 

In [15]:
!pip install rouge-score nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=1eecfc6d84e5134c692bebaf9aff7d2f042da51ca6d8bcee59c68a46ced91b69
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [16]:
import torch
import datasets
import transformers
import evaluate
from pprint import pprint
from rouge_score import rouge_scorer

In [3]:
dataset_name = 'knkarthick/dialogsum'
dataset = datasets.load_dataset(dataset_name)
type(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/12460 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1500 [00:00<?, ? examples/s]

datasets.dataset_dict.DatasetDict

In [4]:
dataset.keys(), dataset.values()

(dict_keys(['train', 'validation', 'test']),
 dict_values([Dataset({
     features: ['id', 'dialogue', 'summary', 'topic'],
     num_rows: 12460
 }), Dataset({
     features: ['id', 'dialogue', 'summary', 'topic'],
     num_rows: 500
 }), Dataset({
     features: ['id', 'dialogue', 'summary', 'topic'],
     num_rows: 1500
 })]))

In [5]:
num_examples = 5

for i in range(num_examples):
  print('-' * 150)
  dialogue = dataset['train'][i]['dialogue']
  summary = dataset['train'][i]['summary']
  topic = dataset['train'][i]['topic']
  print("DIALOGUE:")
  print(dialogue)
  print("\nSUMMARY:")
  print(summary)
  print('\n\n')

------------------------------------------------------------------------------------------------------------------------------------------------------
DIALOGUE:
#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications tha

In [6]:
model_name = 'google/flan-t5-base'
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, use_fast = True)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [7]:
sentence_example = "Hello, How are you?"
tokenized_sentence = tokenizer(sentence_example, return_tensors = 'pt')
tokenized_sentence

{'input_ids': tensor([[8774,    6,  571,   33,   25,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [8]:
decoded_sentence = tokenizer.decode(tokenized_sentence['input_ids'][0])
decoded_sentence

'Hello, How are you?</s>'

## without prompt engineering

In [18]:
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer = True)

for i in range(num_examples):
  dialogue = dataset['test'][i]['dialogue']
  summary = dataset['test'][i]['summary']

  tokenized_dialogue = tokenizer(dialogue, return_tensors = 'pt')
  model_summary_tokens = model.generate(tokenized_dialogue['input_ids'], max_new_tokens = 50)
  decoded_summary = tokenizer.decode(model_summary_tokens[0], skip_special_tokens = True)

  print('-' * 150)
  print("DIALOGUE:")
  print(dialogue)
  print("\nSUMMARY:")
  print(summary)
  print("\nMODEL SUMMARY (WITHOUT PROMPT ENGINEERING):")
  print(decoded_summary)

  rouge_scores = rouge.score(summary, decoded_summary)
  print("\nROUGE Scores:")
  for key, value in rouge_scores.items():
    print(f"{key}: {value}")

  print('\n\n')



------------------------------------------------------------------------------------------------------------------------------------------------------
DIALOGUE:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to c

# prompt engineering

## zero-shot inference

In [19]:
print('*' * 60, ' Zero-shot prompt ', '*' * 60)
for i in range(num_examples):
  dialogue = dataset['test'][i]['dialogue']
  summary = dataset['test'][i]['summary']

  print('-' * 150)
  prompt = f"""
Summarize the following dialogue:
{dialogue}

Summary:

  """

  encoded_prompt = tokenizer(prompt, return_tensors = 'pt')
  model_summary_tokens = model.generate(encoded_prompt['input_ids'], max_new_tokens = 50)
  decoded_summary = tokenizer.decode(model_summary_tokens[0], skip_special_tokens = True)
  print("\nOriginal summary:")
  print(summary)
  print("\nMODEL SUMMARY:")
  print(decoded_summary, '\n')

  rouge_scores = rouge.score(summary, decoded_summary)
  print("\nROUGE Scores:")
  for key, value in rouge_scores.items():
      print(f"{key}: {value}")
  print('\n\n')



************************************************************  Zero-shot prompt  ************************************************************
------------------------------------------------------------------------------------------------------------------------------------------------------

Original summary:
Ms. Dawson helps #Person1# to write a memo to inform every employee that they have to change the communication method and should not use Instant Messaging anymore.

MODEL SUMMARY:
The memo is to be distributed to all employees by this afternoon. 


ROUGE Scores:
rouge1: Score(precision=0.4166666666666667, recall=0.18518518518518517, fmeasure=0.2564102564102564)
rouge2: Score(precision=0.0, recall=0.0, fmeasure=0.0)
rougeL: Score(precision=0.25, recall=0.1111111111111111, fmeasure=0.15384615384615383)



------------------------------------------------------------------------------------------------------------------------------------------------------

Original summary:
In order t

## one-shot inference

In [41]:
def make_prompts(dialogues_indices, dialogue_to_test_index):
  prompt = ''

  for dialogue_index in dialogues_indices:
    dialogue = dataset['test'][dialogue_index]['dialogue']
    summary = dataset['test'][dialogue_index]['summary']
    prompt += f"""
Dialogue:

{dialogue}

What was going on?

{summary}


"""

  dialogue = dataset['test'][dialogue_to_test_index]['dialogue']
  prompt += f"""

{dialogue}

What was going on?

"""
  return prompt

In [42]:
one_shot_prompt = make_prompts([10], 12)
print(one_shot_prompt)


Dialogue:

#Person1#: Happy Birthday, this is for you, Brian.
#Person2#: I'm so happy you remember, please come in and enjoy the party. Everyone's here, I'm sure you have a good time.
#Person1#: Brian, may I have a pleasure to have a dance with you?
#Person2#: Ok.
#Person1#: This is really wonderful party.
#Person2#: Yes, you are always popular with everyone. and you look very pretty today.
#Person1#: Thanks, that's very kind of you to say. I hope my necklace goes with my dress, and they both make me look good I feel.
#Person2#: You look great, you are absolutely glowing.
#Person1#: Thanks, this is a fine party. We should have a drink together to celebrate your birthday

What was going on?

#Person1# attends Brian's birthday party. Brian thinks #Person1# looks great and charming.




#Person1#: This Olympic park is so big!
#Person2#: Yes. Now we are in the Olympic stadium, the center of this park.
#Person1#: Splendid! When is it gonna be finished?
#Person2#: The whole stadium is to be

In [43]:
original_summary = dataset['test'][12]['summary']
print(f"Original summary: {original_summary}")
encoded_prompt = tokenizer(one_shot_prompt, return_tensors = 'pt')
model_summary_tokens = model.generate(encoded_prompt['input_ids'], max_new_tokens = 50)
decoded_summary = tokenizer.decode(model_summary_tokens[0], skip_special_tokens = True)
print(f"Model's summary: {decoded_summary}")

Original summary: #Person1# is surprised at the Olympic Stadium'volume, capacity and interior setting to #Person1#.
Model's summary: The Olympic park is so big that there are 5000 seats in the stand.


## few-shot inference

In [48]:
few_shot_prompt = make_prompts(range(1, 5), 12)
print(few_shot_prompt)


Dialogue:

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with 

In [49]:
original_summary = dataset['test'][12]['summary']
print(f"Original summary: {original_summary}")
encoded_prompt = tokenizer(few_shot_prompt, return_tensors = 'pt')
model_summary_tokens = model.generate(encoded_prompt['input_ids'], max_new_tokens = 50)
decoded_summary = tokenizer.decode(model_summary_tokens[0], skip_special_tokens = True)
print(f"Model's summary: {decoded_summary}")

Original summary: #Person1# is surprised at the Olympic Stadium'volume, capacity and interior setting to #Person1#.
Model's summary: The Olympic park is so big that it is a big stadium.


# Generative configuration parameters for inference

In [55]:
from transformers import GenerationConfig
config = GenerationConfig(max_new_tokens = 50)

In [56]:
original_summary = dataset['test'][12]['summary']
print(f"Original summary: {original_summary}")
encoded_prompt = tokenizer(few_shot_prompt, return_tensors = 'pt')
model_summary_tokens = model.generate(encoded_prompt['input_ids'], generation_config = config)
decoded_summary = tokenizer.decode(model_summary_tokens[0], skip_special_tokens = True)
print(f"Model's summary: {decoded_summary}")

Original summary: #Person1# is surprised at the Olympic Stadium'volume, capacity and interior setting to #Person1#.
Model's summary: The Olympic park is so big that it is a big stadium.s is the center of the park.s is the Olympic stadium.s is the center of the park.s is the Olympic stadium.s is the
